In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [25]:
load_dotenv()

True

In [26]:
template = """
You are an expert SQL query writer. 
Given the database schema and the user's question, write the most accurate SQL query to answer the question. 
Use only the tables and columns mentioned in the schema.

Schema:
{schema}

Question:
{question}

SQL Query:
"""
prompt = ChatPromptTemplate.from_template(template)

In [27]:
prompt.format(schema="myschema", question= "How many users are there?")

"Human: \nYou are an expert SQL query writer. \nGiven the database schema and the user's question, write the most accurate SQL query to answer the question. \nUse only the tables and columns mentioned in the schema.\n\nSchema:\nmyschema\n\nQuestion:\nHow many users are there?\n\nSQL Query:\n"

In [28]:
db_uri = "mysql+mysqlconnector://root:@localhost:3306/chinook"
db = SQLDatabase.from_uri(db_uri)

In [29]:
def get_schema(_):
    return db.get_table_info()

In [30]:
llm = ChatOpenAI()

In [31]:
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema) 
    | prompt 
    | llm.bind(stop="\nSQL Result:") 
    | StrOutputParser()
)

In [32]:
template = """
You are a data analyst. Using the table schema, SQL query, and its result, generate a clear and natural language answer to the user's question.

Schema:
{schema}

Question:
{question}

SQL Query:
{query}

SQL Response:
{response}
"""
prompt = ChatPromptTemplate.from_template(template)

In [35]:
def run_query(query):
    return db.run(query)

In [36]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema, 
        response=lambda vars: run_query(vars["query"])
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [37]:
result = full_chain.invoke({
    "question": "Which three countries have the highest total amounts?"
})


In [38]:
print(result)

The three countries with the highest total amounts from invoices are the USA with a total amount of $523.06, followed by Canada with a total amount of $303.96, and France with a total amount of $195.10.
